In [ ]:
# showcases original graph gen
import math
input_text = """"""

In [ ]:
max_chunck_length = 400
import nltk.data

pkt_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
sentences = pkt_tokenizer.tokenize(input_text)

merged_sentences = []

for i in range(len(sentences)):
    sentence = sentences[i]
    if i >= 1 and len(merged_sentences[-1]) + len(sentence) <= max_chunck_length:
        merged_sentences[-1] += " " + sentence
    else:
        if i >= 1:
            merged_sentences.append(sentences[i - 1])
        merged_sentences.append(sentence)

merged_sentences

In [ ]:
if True:
    from unsloth import FastLanguageModel

    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name="graph_2b_test",
        max_seq_length=2048,
        dtype=None,
        load_in_4bit=True,
    )
    FastLanguageModel.for_inference(model)  # Enable native 2x faster inference
    
prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Extract the most confident information in the sentence below as much as possible, and express the relationships in RDF Triples that complement the existing RDF triples. Do not use information from common sense.
### Input:
{}
### Response:
<T>{}"""

In [ ]:
for chunk in merged_sentences:
    print(f'####proc || {chunk}')
    inputs = tokenizer(
        [
            prompt.format(
                chunk,  # input
                "",  # output - leave this blank for generation!
            )
        ], return_tensors="pt").to("cuda")
    
    outputs = model.generate(
        **inputs, 
        
        # temperature = 0.9,
        # max_new_tokens = max(len(text) + 100, 700), 
        # num_beams = 3,
        # early_stopping = True,
        
        use_cache=True, # Use cache = false is broken haha, but beam search is broken when not using cache hahahah ;-;
    )
    response = tokenizer.batch_decode(outputs)
    print(response)